### Explore medical procedures and costs in healthcare claims data (Medicare, 2012)
### Part 1: Obtaining and arranging data

Janos A. Perge, 08/11/2016

Purpose:   
1) Access data on the number and cost of medical procedures performed by Medicare providers.   
2) Convert HCPCS (or CPT) procedure codes to CCS codes for further analysis.  
3) Save data in three spreadsheets: #1-Provider information, #2-CPT-to-CCS conversion table and #3-Number of performed procedures per provider, broken down to different CCS categories (244 different procedure types).   
4) Explore and visualize this saved data set by running 'visualize_procedures.ipynb'.
    
To run this analysis:  
-Clone this github repository on your hard drive (git@github.com:jperge/CMS_procedures_per_provider.git)

-Get the Medicare-provider-charge data. This is a publicly available file on medical procedures and the associated cost performed by Medicare providers during year 2012. The data is downloadable from CMS:
https://www.cms.gov/apps/ama/license.asp?file=http://download.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Medicare-Provider-Charge-Data/Downloads/Medicare_Provider_Util_Payment_PUF_CY2012_update.zip
Visit the url above, accept the CMS disclaimer, download and unzip file (2GB) and place it within the same directory as this script. The data is described in detail in 'Medicare-Physician-and-Other-Supplier-PUF-Methodology.PDF', included in this repository.  

-All analyzis is written in R. If you are using RStudio, make sure to manually set the working directory to this source file location (under Session/Set Working Directory/To Source File Location

-Note that most of the runtime is spent on opening 2GB of data and dumping it into RAM (~1min on my Win10 machine with core i7 and 8GB RAM) or writing the results to disc (~1 min). The actual processing time of the data on my machine took ~40sec. If you need to run this code repeatedly, uncomment line "save(physician_data, file=my_data_file)", which will save the entire CMS spreadsheet into an .RData file. This file is smaller and faster to read than the original data file.  

To simplify (or compress) the roughly 9000 procedure codes into 244 categories, I convert HCPCS (or CPT) codes to CCS codes using a conversion table available by HCUP (https://www.hcup-us.ahrq.gov). This table is already downloaded in this repository (2016_ccs_services_procedures.csv), but is also available online with further information on the conversion:  
https://www.hcup-us.ahrq.gov/toolssoftware/ccs_svcsproc/ccssvcproc.jsp#info

To obtain information on the provider's graduation year, I use a different data source from CMS (https://data.medicare.gov/data/physician-compare) and merge it into this data set using NPI as a cross link. The extracted graduation years are already included in this repository (physician_grad_year.csv). However, if you wish to repeat the process, the source code is in 'obtain_gradyear.ipynb'.

Big thanks to Vik Paruchuri for providing example code which got me started on this CMS data set!  (http://www.vikparuchuri.com/blog/exploring-us-healthcare-data/). Further analyses on this data set can be also found on Propublica (https://www.propublica.org/series/examining-medicare).

#### Obtain R-packages

In [1]:
rm(list=ls())

##Set working directory:
##If running this script in RStudio:
#setwd(dirname(rstudioapi::getActiveDocumentContext()$path))

##if running this in base R, try:
# File <- "procedures_by_provider.R"
# Files <- list.files(path=file.path("~"),recursive=T,include.dirs=T)
# Path.file <- names(unlist(sapply(Files,grep,pattern=File))[1])
# Dir.wd <- dirname(Path.file)
# setwd(Dir.wd)

packageList = c("data.table","stringr",'plyr')

is_installed <- function(mypkg) is.element(mypkg, installed.packages()[,1])

load_or_install<-function(package_names)
{
  for(package_name in package_names)
  {
    if(!is_installed(package_name))
    {
      install.packages(package_name,repos="http://lib.stat.cmu.edu/R/CRAN")
    }
    options(java.parameters = "-Xmx8g")
    library(package_name,character.only=TRUE,quietly=TRUE,verbose=FALSE)
  }
}

load_or_install(packageList)

#### Access data

In [2]:
cms_filename = "Medicare_Provider_Util_Payment_PUF_CY2012.txt" #data is also available on CMS for years 2013 and 2014
my_data_file = "procedures2012.RData"

start = Sys.time()
#open data from tabular file saved on HD or from Rdatafile:
if(file.exists(my_data_file) && !exists("physician_data")){
  load(my_data_file)
} else if(!file.exists(my_data_file)) {
  #physician_data = read.delim(cms_filename, stringsAsFactors=FALSE)
  physician_data = data.frame(fread(cms_filename)) #This second way of reading data is ~5 times faster!
  physician_data = physician_data[2:nrow(physician_data),]
  colnames(physician_data) = tolower(colnames(physician_data))

  # save(physician_data, file=my_data_file)
}
# physician_data = data.table(physician_data)
    
Sys.time()-start

Time difference of 50.82755 secs

In [3]:
head(physician_data)

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,...,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_medicare_allowed_amt,stdev_medicare_allowed_amt,average_submitted_chrg_amt,stdev_submitted_chrg_amt,average_medicare_payment_amt,stdev_medicare_payment_amt
2,1003000126,ENKESHAFI,ARDALAN,,M.D.,M,I,900 SETON DR,,CUMBERLAND,...,N,115,112,115,135.25,0,199,0,108.11565217,0.9005883395
3,1003000126,ENKESHAFI,ARDALAN,,M.D.,M,I,900 SETON DR,,CUMBERLAND,...,N,93,88,93,198.59,0,291,9.5916630466,158.87,0
4,1003000126,ENKESHAFI,ARDALAN,,M.D.,M,I,900 SETON DR,,CUMBERLAND,...,N,111,83,111,38.75,0,58,0,30.720720721,2.9291057922
5,1003000126,ENKESHAFI,ARDALAN,,M.D.,M,I,900 SETON DR,,CUMBERLAND,...,N,544,295,544,70.95,0,105,0,56.655661765,2.4313271619
6,1003000126,ENKESHAFI,ARDALAN,,M.D.,M,I,900 SETON DR,,CUMBERLAND,...,N,75,55,75,101.74,0,150,0,81.39,0
7,1003000126,ENKESHAFI,ARDALAN,,M.D.,M,I,900 SETON DR,,CUMBERLAND,...,N,95,95,95,71.06,0,104,0,55.768842105,7.4155003896


In [4]:
colnames(physician_data)

[1] "npi"                              "nppes_provider_last_org_name"    
 [3] "nppes_provider_first_name"        "nppes_provider_mi"               
 [5] "nppes_credentials"                "nppes_provider_gender"           
 [7] "nppes_entity_code"                "nppes_provider_street1"          
 [9] "nppes_provider_street2"           "nppes_provider_city"             
[11] "nppes_provider_zip"               "nppes_provider_state"            
[13] "nppes_provider_country"           "provider_type"                   
[15] "medicare_participation_indicator" "place_of_service"                
[17] "hcpcs_code"                       "hcpcs_description"               
[19] "hcpcs_drug_indicator"             "line_srvc_cnt"                   
[21] "bene_unique_cnt"                  "bene_day_srvc_cnt"               
[23] "average_medicare_allowed_amt"     "stdev_medicare_allowed_amt"      
[25] "average_submitted_chrg_amt"       "stdev_submitted_chrg_amt"        
[27] "average_medicare_payment_amt"     "stdev_medicare_payment_amt"

In [5]:
#physician data is large and clogs memory. Therefore I take what I need and clear the rest from the workspace:
descriptor_vars = c("npi", "nppes_provider_last_org_name", "nppes_provider_first_name", "nppes_provider_mi", 
                    "nppes_credentials", "nppes_provider_gender", "nppes_entity_code", "nppes_provider_street1", 
                    "nppes_provider_street2", "nppes_provider_city", "nppes_provider_zip", "nppes_provider_state", 
                    "nppes_provider_country", "provider_type", "medicare_participation_indicator", "place_of_service")

physician_info   = physician_data[, descriptor_vars]
doctor_procedure = physician_data[, c('npi', 'hcpcs_code','line_srvc_cnt',"bene_unique_cnt", "bene_day_srvc_cnt", 
                                      "average_medicare_allowed_amt", "stdev_medicare_allowed_amt", 
                                      "average_submitted_chrg_amt", "stdev_submitted_chrg_amt", "average_medicare_payment_amt",
                                      "stdev_medicare_payment_amt")]
conversion_table = physician_data[, c('hcpcs_code', 'hcpcs_description')]
rm(physician_data)

#### Spreadsheet #1: Doctor's parameters such as NPI, Name, state, gender, etc...

In [6]:
#bring in physician years of expertise:
npi_file = 'physician_grad_year.csv'
npi_frame = data.frame(fread(npi_file))
colnames(npi_frame) = tolower(colnames(npi_frame))
head(npi_frame)

,npi,medical.school.name,graduation.year
1,1003000126,OTHER,1994
2,1003000134,UNIVERSITY OF KENTUCKY COLLEGE OF MEDICINE,2003
3,1003000142,OTHER,1999
4,1003000407,PHILADELPHIA COLLEGE OF OSTEOPATHIC MEDICINE,2003
5,1003000415,OTHER,2005
6,1003000423,TOLEDO MEDICAL COLLEGE,2007


In [7]:
physician_info = data.table(physician_info)
setkey(physician_info, npi)
physician_info = unique(physician_info)
physician_info = merge(physician_info, npi_frame, all.x=TRUE) #left outer join

In [8]:
head(physician_info)

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip,nppes_provider_state,nppes_provider_country,provider_type,medicare_participation_indicator,place_of_service,medical.school.name,graduation.year
1,1003000126,ENKESHAFI,ARDALAN,,M.D.,M,I,900 SETON DR,,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,OTHER,1994
2,1003000134,CIBULL,THOMAS,L,M.D.,M,I,2650 RIDGE AVE,EVANSTON HOSPITAL,EVANSTON,602011718,IL,US,Pathology,Y,F,UNIVERSITY OF KENTUCKY COLLEGE OF MEDICINE,2003
3,1003000142,KHALIL,RASHID,,M.D.,M,I,4126 N HOLLAND SYLVANIA RD,SUITE 220,TOLEDO,436233536,OH,US,Anesthesiology,Y,O,OTHER,1999
4,1003000381,BRAGANZA,LUTHER,Q,PT,M,I,134 N OLD DIXIE HWY,,LADY LAKE,321594347,FL,US,Physical Therapist,Y,O,NA,NA
5,1003000407,GIRARDI,DAVID,J,D.O.,M,I,456 MAGEE AVE,,PATTON,166681219,PA,US,Family Practice,Y,F,PHILADELPHIA COLLEGE OF OSTEOPATHIC MEDICINE,2003
6,1003000423,VELOTTA,JENNIFER,A,M.D.,F,I,11100 EUCLID AVE,,CLEVELAND,441061716,OH,US,Obstetrics/Gynecology,Y,O,TOLEDO MEDICAL COLLEGE,2007


#### Spreadsheet #2: HCPCS/CPT code to CCS conversion

In [9]:
conversion_table = conversion_table[!duplicated(conversion_table$hcpcs_code),]
conversion_table = data.table(conversion_table)
setkey(conversion_table, hcpcs_code)

In [10]:
head(conversion_table)

,hcpcs_code,hcpcs_description
1,00100,Anesthesia for procedure on salivary gland with biopsy
2,00102,Anesthesia for procedure to repair lip defect present at birth
3,00103,Anesthesia for procedure on eyelid
4,00104,Anesthesia for electric shock treatment
5,00120,Anesthesia for biopsy of external middle and inner ear
6,00126,Anesthesia for incision of ear drum


In [11]:
ccs_file = '2016_ccs_services_procedures.csv'
ccs_table = data.table(read.csv(ccs_file))
ccs_table$Code.Range <- as.character(ccs_table$Code.Range)
setkey(ccs_table,Code.Range,CCS,CCS.Label)
head(ccs_table)

,Code.Range,CCS,CCS.Label
1,'0001T-0002T',52,"Aortic resection, replacement or anastomosis"
2,'0003T-0003T',130,"Other diagnostic procedures, female organs"
3,'0005T-0006T',59,Other OR procedures on vessels of head and neck
4,'0007T-0007T',211,Therapeutic radiology
5,'0008T-0008T',93,Other non-OR upper GI therapeutic procedures
6,'0009T-0009T',125,Other excision of cervix and uterus


In [12]:
# combine the above two tables

In [13]:
#create an incremental sequence of CSS codes from Code.Range:
get_code_range <- function(inp,ccscode,ccsdesc){
    
    code_range <- vector(mode="numeric", length=0) #empty 
    
    aaa = unlist(strsplit(inp, "-", fixed = TRUE))    
    aaa = sub("\'", "", aaa)
    
    if (!grepl("[a-zA-Z]", aaa[1])){  #if code does not contain letters
        aaa = as.numeric(aaa)
        code_range = seq.int(aaa[1],aaa[2])
        code_range = sprintf("%05d", code_range) # fixed width of five characters with leading zeros
        code_range = as.character(code_range)
        
    } else #if hcpcs code is alphanumeric, with the numeric part as an incremental sequence
    {
        bbb = substring(aaa[1], seq(1,nchar(aaa[1])), seq(1,nchar(aaa[1]),1)) #break up string to individual characters
        letterPos = grep("[a-zA-Z]", bbb, value = FALSE)
        letterChar = grep("[a-zA-Z]", bbb, value = TRUE)

        numericPart1 = grep("[0-9]", bbb, value = TRUE)
        numericPart1 = as.numeric(paste(numericPart1, collapse=""))

        bbb = substring(aaa[2], seq(1,nchar(aaa[2])), seq(1,nchar(aaa[2]),1)) 
        numericPart2 = grep("[0-9]", bbb, value = TRUE)
        numericPart2 = as.numeric(paste(numericPart2, collapse=""))

        cr  = seq.int(numericPart1,numericPart2)
        
        if (letterPos==1){
            code_range = sprintf("%s%04d", letterChar, cr) # fixed width of four characters with leading zeros
        } else
        {
            code_range = sprintf("%04d%s", cr, letterChar)
        }            
    }
    out = data.frame(hcpcs_code=code_range, ccs_code = ccscode, css_desc = ccsdesc)
}

In [14]:
# Expand CCS table, i.e. list every HCPCS code specified within the Code.Range
expanded_ccs = with(ccs_table, Map(get_code_range, Code.Range, CCS, CCS.Label))
expanded_ccs = rbind.fill(expanded_ccs)
expanded_ccs = data.table(expanded_ccs) 
setkey(expanded_ccs,hcpcs_code)
conversion_table = merge(conversion_table,expanded_ccs, by='hcpcs_code')

In [15]:
head(conversion_table)

,hcpcs_code,hcpcs_description,ccs_code,css_desc
1,00100,Anesthesia for procedure on salivary gland with biopsy,232,Anesthesia
2,00102,Anesthesia for procedure to repair lip defect present at birth,232,Anesthesia
3,00103,Anesthesia for procedure on eyelid,232,Anesthesia
4,00104,Anesthesia for electric shock treatment,232,Anesthesia
5,00120,Anesthesia for biopsy of external middle and inner ear,232,Anesthesia
6,00126,Anesthesia for incision of ear drum,232,Anesthesia


#### Spreadsheet #3: Provider vs Procedure count.  
cells in the final matrix (npi_vs_css) correspond to total procedure counts for a given provider and given CCS category

In [16]:
# Merge CCS codes into physician data frame:
toAppend = conversion_table[, .(hcpcs_code ,ccs_code)]
setkey(toAppend,hcpcs_code)

In [17]:
head(toAppend)

,hcpcs_code,ccs_code
1,00100,232
2,00102,232
3,00103,232
4,00104,232
5,00120,232
6,00126,232


In [18]:
doctor_procedure = data.table(doctor_procedure)
setkey(doctor_procedure, "hcpcs_code")
doctor_procedure = merge(doctor_procedure, toAppend, by='hcpcs_code')

In [19]:
head(doctor_procedure)

,hcpcs_code,npi,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_medicare_allowed_amt,stdev_medicare_allowed_amt,average_submitted_chrg_amt,stdev_submitted_chrg_amt,average_medicare_payment_amt,stdev_medicare_payment_amt,ccs_code
1,00100,1396897104,14,12,14,125.05928571,27.539691217,1401.4285714,305.13643653,100.04857143,22.032023076,232
2,00100,1508884156,12,12,12,353.00416667,80.615788513,2122.5,652.5862012,282.4025,64.492407838,232
3,00100,1528032067,14,14,14,185.57,27.338754805,572.92857143,84.316126475,148.45642857,21.87183205,232
4,00100,1780646562,13,13,13,357.89923077,198.10791711,2098.4615385,965.20899414,286.32076923,158.48520443,232
5,00102,1700846235,17,17,17,200.42941176,52.47849223,1569.7058824,827.42597072,152.30705882,43.448522044,232
6,00103,1003019191,14,14,14,90.253571429,15.035313008,393.53714286,229.38057009,65.611428571,13.137521976,232


In [20]:
#pool similar procedures (with identical ccs codes) per provider
setkey(doctor_procedure, "npi","ccs_code")
#doctor_procedure = doctor_procedure[,proc_per_patient := line_srvc_cnt/bene_unique_cnt]
doctor_procedure = doctor_procedure[,proc_per_patient := bene_day_srvc_cnt/bene_unique_cnt]
npi_vs_ccs = doctor_procedure[, .(proc_cnt=sum(line_srvc_cnt, na.rm=T), 
                                 uniq_cnt=sum(bene_unique_cnt, na.rm=T),
                                 day_srvc_cnt=sum(bene_day_srvc_cnt, na.rm=T),
                                 bene_cnt=sum(bene_unique_cnt, na.rm=T),
                                 med_proc_per_patient=median(proc_per_patient, na.rm=T), 
                                 est_allo_amt=sum(average_medicare_allowed_amt*line_srvc_cnt, na.rm=T), 
                                 est_pay_amt=sum(average_medicare_payment_amt*line_srvc_cnt, na.rm=T)), 
                                 by = .(npi,ccs_code)] 
# setkey(npi_vs_ccs, "npi","ccs_code")
# npi_vs_ccs= dcast(npi_vs_ccs, npi ~ ccs_code) #convert data.table from long to wide format (i.e. npi vs. ccs table)

In [21]:
tail(npi_vs_ccs)

,npi,ccs_code,proc_cnt,uniq_cnt,day_srvc_cnt,bene_cnt,med_proc_per_patient,est_allo_amt,est_pay_amt,na.rm
1,1.993000e+09,2.600000e+01,6.400000e+01,6.000000e+01,6.400000e+01,6.000000e+01,1.066667e+00,3.509100e+03,2.675520e+03,1.000000e+00
2,1.993000e+09,3.500000e+01,7.200000e+01,6.100000e+01,7.200000e+01,6.100000e+01,1.155968e+00,8.075370e+03,6.432890e+03,1.000000e+00
3,1.993000e+09,2.270000e+02,6.110000e+02,5.300000e+02,6.110000e+02,5.300000e+02,1.045455e+00,7.495100e+04,5.594215e+04,1.000000e+00
4,1.993e+09,2.310e+02,2.200e+01,2.200e+01,2.200e+01,2.200e+01,1.000e+00,6.600e+01,6.600e+01,1.000e+00
5,1.9930e+09,2.3300e+02,2.3000e+01,2.3000e+01,2.3000e+01,2.3000e+01,1.0000e+00,1.6698e+02,1.6698e+02,1.0000e+00
6,1.993000e+09,2.270000e+02,9.940000e+02,6.420000e+02,9.940000e+02,6.420000e+02,1.027395e+00,9.589056e+04,7.654505e+04,1.000000e+00


In [22]:
#These procedure codes will be used to estimate the average length of outpatient office visits:
sub = conversion_table[hcpcs_code %in% c('99211', '99212', '99213', '99214', '99215')]
sub

,hcpcs_code,hcpcs_description,ccs_code,css_desc
1,99211,"Established patient office or other outpatient visit, typically 5 minutes",227,"Other diagnostic procedures (interview, evaluation, consultation)"
2,99212,"Established patient office or other outpatient visit, typically 10 minutes",227,"Other diagnostic procedures (interview, evaluation, consultation)"
3,99213,"Established patient office or other outpatient visit, typically 15 minutes",227,"Other diagnostic procedures (interview, evaluation, consultation)"
4,99214,"Established patient office or other outpatient, visit typically 25 minutes",227,"Other diagnostic procedures (interview, evaluation, consultation)"
5,99215,"Established patient office or other outpatient, visit typically 40 minutes",227,"Other diagnostic procedures (interview, evaluation, consultation)"


In [23]:
#calculate the average duration of office visits per provider and merge it into physician_info table
#use hcpcs codes 99211-215 to estimate office visit durations
#Half of the providers (~400,000) have these office visit codes

setkey(doctor_procedure,hcpcs_code)
office_visits = doctor_procedure[hcpcs_code %in% c('99211', '99212', '99213', '99214', '99215')]
setkey(office_visits, hcpcs_code)
office_visits = office_visits[,cpt := as.numeric(as.character(hcpcs_code))]

setkey(office_visits, cpt)
office_visits = office_visits[cpt==99211,visit_dur := 5]
office_visits = office_visits[cpt==99212,visit_dur := 10]
office_visits = office_visits[cpt==99213,visit_dur := 15]
office_visits = office_visits[cpt==99214,visit_dur := 25]
office_visits = office_visits[cpt==99215,visit_dur := 40]

office_visits = office_visits[, tot_office_mins:= visit_dur*line_srvc_cnt]
setkey(office_visits, npi)
office_per_doc = office_visits[,.(office_mins = sum(tot_office_mins,na.rm=T)/sum(line_srvc_cnt,na.rm=T)), by=npi]

#merge this into physician info table:
physician_info = merge(physician_info, office_per_doc, all.x=TRUE)

#### Save results

In [24]:
#csv files:
start = Sys.time()
# write.csv(physician_info, file = "physician_info.csv", row.names=FALSE, na="")
# write.csv(conversion_table, file = "CPT_to_CCS_conversion.csv", row.names=FALSE, na="")
# write.csv(npi_vs_ccs, file = "provider_vs_procedures2012.csv", row.names=FALSE, na="")

#Note that write.csv saves numeric hcpcs codes as an arabic number e.g. '102', omiting zero characters in the beginning.
#This would cause mismatches later when mapping fixed-length character arrays such as '00102'. 
#However, R's native data format saves the codes with leading zeros.

#R.data file:
save(physician_info, conversion_table, npi_vs_ccs, file='provider_vs_procedures_2_2012.RData')

Sys.time()-start

Time difference of 27.56068 secs

In [25]:
#rm(list=ls())